In [33]:
import tensorflow as tf
from tensorflow.keras.datasets import mnist
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Reshape, Flatten
from tensorflow.keras.layers import Conv2D, Conv2DTranspose
from tensorflow.keras.layers import LeakyReLU, Dropout
from tensorflow.keras.optimizers import Adam
import numpy as np
import matplotlib.pyplot as plt

# Define the generator model
def build_generator():
    model = Sequential()
    model.add(Dense(7*7*256, input_dim=100))
    model.add(LeakyReLU(0.2))
    model.add(Reshape((7, 7, 256)))
    model.add(Conv2DTranspose(128, kernel_size=5, strides=1, padding='same'))
    model.add(LeakyReLU(0.2))
    model.add(Conv2DTranspose(64, kernel_size=5, strides=2, padding='same'))
    model.add(LeakyReLU(0.2))
    model.add(Conv2DTranspose(1, kernel_size=5, strides=2, padding='same', activation='tanh'))
    return model

# Define the discriminator model
def build_discriminator():
    model = Sequential()
    model.add(Conv2D(64, kernel_size=5, strides=2, padding='same', input_shape=(28, 28, 1)))
    model.add(LeakyReLU(0.2))
    model.add(Dropout(0.3))
    model.add(Conv2D(128, kernel_size=5, strides=2, padding='same'))
    model.add(LeakyReLU(0.2))
    model.add(Dropout(0.3))
    model.add(Flatten())
    model.add(Dense(1, activation='sigmoid'))
    return model

# Combine the generator and discriminator into a GAN model
def build_gan(generator, discriminator):
    discriminator.trainable = False
    model = Sequential()
    model.add(generator)
    model.add(discriminator)
    return model

# Define the loss functions and optimizers
cross_entropy = tf.keras.losses.BinaryCrossentropy(from_logits=True)
generator_optimizer = Adam(learning_rate=0.0002)
discriminator_optimizer = Adam(learning_rate=0.0002)

# Initialize the models
generator = build_generator()
discriminator = build_discriminator()
gan = build_gan(generator, discriminator)

# Load and preprocess the dataset
(x_train, _), (_, _) = mnist.load_data()
x_train = x_train.reshape(x_train.shape[0], 28, 28, 1).astype('float32')
x_train = (x_train - 127.5) / 127.5

# Set the batch size and number of epochs
batch_size = 64
epochs = 10
steps_per_epoch = x_train.shape[0] // batch_size

# Generate and save sample images during training
def generate_images(generator, epoch, latent_dim, examples=100):
    random_latent_vectors = np.random.normal(size=(examples, latent_dim))
    generated_images = generator.predict(random_latent_vectors)
    generated_images = 0.5 * generated_images + 0.5
    fig, axs = plt.subplots(examples // 10, 10, figsize=(10, examples // 10))
    for i in range(examples):
        ax = axs[i // 10, i % 10]
        ax.imshow(generated_images[i, :, :, 0], cmap='gray')
        ax.axis('off')
    plt.tight_layout()
    plt.savefig(f'generated_images_epoch_{epoch}.png')
    plt.close()

# Define the training loop
def train(generator, discriminator, gan, epochs, batch_size, steps_per_epoch):
    latent_dim = 100

    for epoch in range(epochs):
        for step in range(steps_per_epoch):
            # Generate random noise samples as input to the generator
            random_latent_vectors = np.random.normal(size=(batch_size, latent_dim))

            # Generate fake images using the generator
            generated_images = generator.predict(random_latent_vectors)

            # Select a random batch of real images from the dataset
            real_images = x_train[np.random.randint(0, x_train.shape[0], size=batch_size)]

            # Concatenate the real and fake images
            combined_images = np.concatenate([generated_images, real_images])

            # Create labels for the generated and real images
            labels = np.concatenate([np.ones((batch_size, 1)), np.zeros((batch_size, 1))])

            # Add noise to the labels
            labels += 0.05 * np.random.random(labels.shape)

            # Train the discriminator
            with tf.GradientTape() as tape:
                predictions = discriminator(combined_images)
                discriminator_loss = cross_entropy(labels, predictions)
            grads = tape.gradient(discriminator_loss, discriminator.trainable_weights)
            discriminator_optimizer.apply_gradients(zip(grads, discriminator.trainable_weights))

            # Generate new random noise samples
            random_latent_vectors = np.random.normal(size=(batch_size, latent_dim))

            # Create labels for the generated images (pretend they are real)
            misleading_labels = np.zeros((batch_size, 1))

            # Train the generator (via the whole GAN model)
            with tf.GradientTape() as tape:
                predictions = gan(random_latent_vectors)
                generator_loss = cross_entropy(misleading_labels, predictions)
            grads = tape.gradient(generator_loss, generator.trainable_weights)
            generator_optimizer.apply_gradients(zip(grads, generator.trainable_weights))

        # Print the losses
        print(f"Epoch {epoch+1}/{epochs}, Discriminator Loss: {discriminator_loss:.4f}, Generator Loss: {generator_loss:.4f}")

        # Generate and save sample images
        generate_images(generator, epoch, latent_dim)

# Train the GAN model
train(generator, discriminator, gan, epochs, batch_size, steps_per_epoch)


2/2 [==============================] - 1s 10ms/step


C:\Users\Josep\anaconda3\envs\tf210\lib\site-packages\keras\backend.py:5673: UserWarning: "`binary_crossentropy` received `from_logits=True`, but the `output` argument was produced by a Sigmoid activation and thus does not represent logits. Was this intended?
  output, from_logits = _get_logits(


2/2 [==============================] - 0s 7ms/step


2/2 [==============================] - 0s 11ms/step


2/2 [==============================] - 0s 9ms/step


...

2/2 [==============================] - 0s 10ms/step


2/2 [==============================] - 0s 10ms/step


2/2 [==============================] - 0s 10ms/step


2/2 [==============================] - 0s 10ms/step


2/2 [==============================] - 0s 9ms/step
Epoch 10/10, Discriminator Loss: 1.0858, Generator Loss: 0.2724
4/4 [==============================] - 0s 10ms/step
